# AI Underwriter Assistant

Author: https://github.com/deburky

In [1]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()

openai_key = os.getenv('OPENAI_API_KEY')

## Prepare training data

Loan approval dataset is used to train the model with a few-shot learning.

In [8]:
# Fetch loan approval data
import pandas as pd
url = (
    "https://drive.google.com/file/d/1r6gGvL_s313ThGSU7ziZiuYr2G_yijaZ/view?usp=sharing"
)
file_id = url.split("/")[-2]
dataset = pd.read_csv(f"https://drive.google.com/uc?id={file_id}")

dataset.drop(columns=['Gender']).sample(n=700).to_csv('loans-9309cbc146a4.csv', index=False)
dataset.drop(columns=['Gender', 'Loan_Status']).sample(n=100).to_csv('loans-new-9309cbc146a4.csv', index=False)

train_set = pd.read_csv("loans-new-9309cbc146a4.csv")
display(train_set.sample(5))

,Loan_ID,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
2,LP001698,No,0,Graduate,Yes,2971,0.0,170.0,360.0,1.0,Semiurban
10,LP001385,No,0,Not Graduate,Yes,2679,1451.0,105.0,360.0,1.0,Semiurban
80,LP002284,Yes,0,Graduate,Yes,7333,1459.0,180.0,360.0,1.0,Urban
66,LP002205,Yes,0,Graduate,Yes,4310,923.0,124.0,360.0,1.0,Urban
56,LP002342,Yes,2,Graduate,Yes,4566,0.0,100.0,360.0,1.0,Semiurban


## Assistant

[OpenAI - Assistants API quickstart](https://platform.openai.com/docs/assistants/quickstart)

[Fun with Assistants – getting them to really talk to each other](https://community.openai.com/t/fun-with-assistants-getting-them-to-really-talk-to-each-other/784073/12)

In [40]:
from openai import OpenAI
client = OpenAI()

# Upload a file with loans data
loan_file = client.files.create(
    file=open("loans-9309cbc146a4.csv", "rb"),
    purpose='assistants'
)

loan_file_new = client.files.create(
    file=open("loans-new-9309cbc146a4.csv", "rb"),
    purpose='assistants'
)

# Create an assistant for AI underwriting
assistant = client.beta.assistants.create(
    name="AI Underwriter Assistant",
    description="""
        You are an AI assistant specialized in underwriting consumer loan applications. 
        You evaluate loan applications based on provided information and make decisions accordingly.
        """
    ,
    model="gpt-4o",  # gpt-4-1106-preview"
    tools=[{"type": "code_interpreter"}],
    tool_resources={
        "code_interpreter": {
        "file_ids": [loan_file.id]
        }
    }
)

## Define the prompt for evaluation

In [41]:
# Define the prompt
prompt = """
    I want you to first familiarize yourself with the past decisions in the first file.
    After understanding the logic of past decisions, I want you to evaluate the decisions in the second file.
    Your job is to give an answer "Y" (yes) or "N" (no) for each decision in the second file.
    Like: Application 1. Decision: Yes. And so on.
    In the end, please prepare a csv with Loan ID and Loan_Status column that you fill in for the second file.
"""

# Create a thread with both messages and tool_resources
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
    tool_resources={  
        "code_interpreter": {  
            "file_ids": [loan_file_new.id]
        }
    }
)

# Run the assistant
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [ ]:
import time
from rich.console import Console
from rich.live import Live
from rich.table import Table

console = Console()

console.print(f"🔄 [cyan1]Run started... (Run ID: {run.id})[/cyan1]")

# Function to fetch messages and display them
def fetch_messages():
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    table = Table(title="Assistant Conversation", show_header=True, header_style="bold cyan1")
    table.add_column("Role", style="cyan1", width=12)
    table.add_column("Message", style="white")

    for message in reversed(messages.data):  # Reverse to show latest first
        text = message.content[0].text.value if message.content else "[No content]"
        table.add_row(message.role, text)

    return table

# Monitor the run until completion
with Live(fetch_messages(), refresh_per_second=1, console=console) as live:
    while True:
        run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run_status.status in ["completed", "failed", "cancelled"]:
            break
        live.update(fetch_messages())
        time.sleep(2)

# Final message update
console.print(fetch_messages())
console.print("✅ [cyan1]Run completed![/cyan1]")

🔄 Run started... (Run ID: run_igGsyLa04dpyO7E8svab4cYX)

/Users/deburky/Library/Caches/pypoetry/virtualenvs/language-models-qT78hDLD-py3.10/lib/python3.10/site-packages/ric
h/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

                                              Assistant Conversation                                               
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Role         ┃ Message                                                                                          ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user         │                                                                                                  │
│              │     I want you to first familiarize yourself with the past decisions in the first file.          │
│              │     After understanding the logic of past decisions, I want you to evaluate the decisions in the │
│              │ second file.                                                                                     │
│              │     Your job is to give an answer "Y" (yes) or "N" (no) for each decision in the second file.    │
│              │     Like: Application 1. Decision: Yes. And so on.                                               │
│              │     In the end, please prepare a csv with Loan ID and Loan_Status column that you fill in for    │
│              │ the second file.                                                                                 │
│              │                                                                                                  │
│ assistant    │ To assist you effectively, let me first inspect the contents of the files you've uploaded. I'll  │
│              │ start by examining the first file to understand the decisions it contains. Then, I'll move on to │
│              │ the second file to evaluate the decisions based on the understanding gained from the first file. │
│              │ Let's start with the first file.                                                                 │
│ assistant    │ The first file has the following columns which are likely used to make loan decisions:           │
│              │                                                                                                  │
│              │ - `Loan_ID`: Unique identifier for each loan application.                                        │
│              │ - `Married`: Marital status of the applicant.                                                    │
│              │ - `Dependents`: Number of dependents.                                                            │
│              │ - `Education`: Education level of the applicant.                                                 │
│              │ - `Self_Employed`: Whether the applicant is self-employed.                                       │
│              │ - `ApplicantIncome`: Income of the applicant.                                                    │
│              │ - `CoapplicantIncome`: Income of the co-applicant.                                               │
│              │ - `LoanAmount`: Loan amount requested.                                                           │
│              │ - `Loan_Amount_Term`: Term of the loan in months.                                                │
│              │ - `Credit_History`: Credit history (1.0 good, 0.0 bad).                                          │
│              │ - `Property_Area`: Area where the property is located.                                           │
│              │ - `Loan_Status`: Approval status of the loan (Y for Yes, N for No).                              │
│              │                                                                                                  │
│              │ The `Loan_Status` column is what we'll focus on to recognize patterns in the approved or denied  │
│              │ loans. Now, let's examine the second file, where we need to make decisions based on the pattern  │
│              │ recognized.                            

✅ Run completed!

## Save report

In [44]:
import pandas as pd
from openai import OpenAI

# Fetch the list of files
files = client.files.list()

# Extract filenames and file IDs
file_info = [(file.id, file.filename, file.purpose, file.status) for file in files.data]

# Convert to DataFrame for better readability
df_files = pd.DataFrame(file_info, columns=["File ID", "Filename", "Purpose", "Status"])

# # Display the extracted filenames
display(df_files)

,File ID,Filename,Purpose,Status
0,file-BNWhQxwMJ8iAddiPexp7Cu,/mnt/data/loan_decisions.csv,assistants_output,processed
1,file-Y7gF6oKQ9eudKd4ryv73qG,loans-new-9309cbc146a4.csv,assistants,processed
2,file-5G7ewhxpp2RhgX5wikX6qi,loans-9309cbc146a4.csv,assistants,processed


In [ ]:
import io
evaluation_results = df_files.iloc[0]['File ID'] # Fetch assistants_output
content = client.files.retrieve_content(evaluation_results)
csv_string = content
csv_file = io.StringIO(csv_string)
df = pd.read_csv(csv_file)
print(df.head(5))

/var/folders/k_/yz8rvp25185_js60dw8vhnj40000gn/T/ipykernel_41203/1370021253.py:3: DeprecationWarning: The `.content()` method should be used instead
  content = client.files.retrieve_content(evaluation_results)


    Loan_ID Loan_Status
0  LP002501           Y
1  LP002530           N
2  LP001698           Y
3  LP001006           Y
4  LP001357           Y


In [52]:
df.to_csv('loans-new-FKJESKpN5LYuSG21np1PYE.csv', index=False)

## Evaluation against ground truth

In [53]:
# Get real decisions outcomes
testing_df = df.merge(
    dataset[['Loan_ID', 'Loan_Status']], 
    on='Loan_ID',
    suffixes=('_Assistant', '_Truth')
)
testing_df.replace({'Y': 1, 'N': 0}, inplace=True)

/var/folders/k_/yz8rvp25185_js60dw8vhnj40000gn/T/ipykernel_41203/2525230116.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  testing_df.replace({'Y': 1, 'N': 0}, inplace=True)


In [54]:
testing_df[['Loan_Status_Assistant', 'Loan_Status_Truth']].mean()

Loan_Status_Assistant    0.812439
Loan_Status_Truth        0.691448
dtype: float64

In [55]:
decision_matrix = pd.crosstab(testing_df['Loan_Status_Truth'], testing_df['Loan_Status_Assistant'], normalize='all')
display(decision_matrix)

Loan_Status_Assistant,0,1
Loan_Status_Truth,,
0,0.063168,0.245384
1,0.124393,0.567055


## Delete the assistant and files

In [ ]:
# Fetch all assistants
assistants = client.beta.assistants.list()

# Delete each assistant
for assistant in assistants.data:
    print(f"Deleting Assistant: {assistant.name} (ID: {assistant.id})")
    client.beta.assistants.delete(assistant.id)

print("✅ All assistants deleted!")

# Fetch all files
files = client.files.list()

# Delete each file
for file in files.data:
    print(f"Deleting File: {file.filename} (ID: {file.id})")
    client.files.delete(file.id)

print("✅ All files deleted!")

✓ All assistants deleted!
✓ All files deleted!
